# VAE

> Scripts to use Variational Autoencoders

In [7]:
#| default_exp vae

In [8]:
#| export
#| hide

import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torchmetrics import MetricCollection, MeanMetric
import torch
import torch.nn as nn
from abc import ABC, abstractmethod

In [9]:
#| export
class Sampling(nn.Module):
    """
    PyTorch module to perform the reparameterization trick for VAEs.
    """
    def forward(self, z_mean, z_log_var):
        std = torch.exp(0.5 * z_log_var)
        eps = torch.randn_like(std)
        return z_mean + eps * std

In [10]:
#| export
class AbstractVAE(ABC):
    def __init__(self, seq_len, feat_dim, latent_dim):
        """
        Initialize the VAE with given sequence length, feature dimension, and latent dimension.

        :param seq_len: The length of the input sequences
        :param feat_dim: The dimensionality of the input features
        :param latent_dim: The dimensionality of the latent space
        """
        self.seq_len = seq_len
        self.feat_dim = feat_dim
        self.latent_dim = latent_dim

    @abstractmethod
    def _reparameterize(self, mu, logvar):
        """
        Abstract method for the reparameterization trick.

        :param mu: Mean from the encoder's latent space
        :param logvar: Log variance from the encoder's latent space
        :return: Sampled latent vector
        """
        pass

    @abstractmethod
    def forward(self, x):
        """
        Abstract method to define the forward pass of the VAE.

        :param x: Input data
        :return: Reconstructed input, latent representation, and other necessary outputs
        """
        pass

In [11]:
#| export
class BetaVAE(pl.LightningModule, AbstractVAE):
    """
    Beta Variational Autoencoder implementation with configurable encoder and decoder.
    Args:
        encoder (nn.Module): The encoder network.
        decoder (nn.Module): The decoder network.
        beta (float): The beta coefficient for the KL divergence term.
        ...
    """
    def __init__(self, encoder, decoder, beta=1.0, loss_fn=None, optimizer_cls=torch.optim.Adam, lr=0.001, **kwargs):
        super().__init__(**kwargs)
        self.beta = beta
        self.encoder = encoder
        self.decoder = decoder
        self.loss_fn = loss_fn if loss_fn else self.default_loss_fn
        self.optimizer_cls = optimizer_cls
        self.lr = lr
        self.metrics = MetricCollection({
            'total_loss': MeanMetric(),
            'reconstruction_loss': MeanMetric(),
            'kl_loss': MeanMetric()
        })
        self.latent_dim = encoder.latent_dim
        self.feat_dim = encoder.feat_dim
        self.seq_len = encoder.seq_len

    def forward(self, x):
        z, z_mean, z_log_var = self.encoder(x)
        return self.decoder(z)


    def _reparameterize(self, mean, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mean + eps * std

    def reconstruction_loss_by_axis(self, x, x_hat, axis):
        # Compute mean squared error along the specified axis
        loss = torch.mean((x - x_hat) ** 2, dim=axis)
        # Sum the loss to get a scalar value
        return loss.sum()

    def default_loss_fn(self, x, x_hat, z_mean, z_log_var):
        # Compute reconstruction loss over each axis and sum them
        reconst_loss = (
            self.reconstruction_loss_by_axis(x, x_hat, axis=0) +
            self.reconstruction_loss_by_axis(x, x_hat, axis=1) +
            self.reconstruction_loss_by_axis(x, x_hat, axis=2)
        )
        # KL Divergence Loss
        kl_loss = -0.5 * torch.mean(
            torch.sum(1 + z_log_var - z_mean ** 2 - torch.exp(z_log_var), dim=1)
        )
        return reconst_loss, self.beta * kl_loss


    def training_step(self, batch, batch_idx):
        x = batch[0] if isinstance(batch, list) else batch
        z, z_mean, z_log_var = self.encoder(x)  # Unpack all three values
        x_hat = self.decoder(z)
        reconst_loss, kl_loss = self.loss_fn(x, x_hat, z_mean, z_log_var)
        total_loss = reconst_loss + kl_loss

        # Update metrics
        self.metrics['total_loss'].update(total_loss)
        self.metrics['reconstruction_loss'].update(reconst_loss)
        self.metrics['kl_loss'].update(kl_loss)

        # Log metrics
        self.log('train_loss', total_loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_reconstruction_loss', reconst_loss, on_step=True, on_epoch=True, prog_bar=False, logger=True)
        self.log('train_kl_loss', kl_loss, on_step=True, on_epoch=True, prog_bar=False, logger=True)
        self.log_dict({f'train_{k}': v.compute() for k, v in self.metrics.items()}, on_step=True, on_epoch=True)

        return total_loss
    
    def validation_step(self, batch, batch_idx):
        x = batch[0] if isinstance(batch, list) else batch
        z, z_mean, z_log_var = self.encoder(x)  # Unpack all three values
        x_hat = self.decoder(z)
        reconst_loss, kl_loss = self.loss_fn(x, x_hat, z_mean, z_log_var)
        total_loss = reconst_loss + kl_loss
        
        # Update validation metrics
        self.val_metrics['total_loss'].update(total_loss)
        self.val_metrics['reconstruction_loss'].update(reconst_loss)
        self.val_metrics['kl_loss'].update(kl_loss)

        # Log metrics
        self.log('val_loss', total_loss, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_reconstruction_loss', reconst_loss, on_epoch=True, prog_bar=False, logger=True)
        self.log('val_kl_loss', kl_loss, on_epoch=True, prog_bar=False, logger=True)
        self.log_dict({f'val_{k}': v.compute() for k, v in self.val_metrics.items()}, on_step=False, on_epoch=True)

        return total_loss

    def configure_optimizers(self):
        return self.optimizer_cls(self.parameters(), lr=self.lr)

    def generate(self, n=1):
        z = torch.randn(n, self.latent_dim, device=self.device)
        return self.decoder(z)

In [12]:
#| hide
import nbdev; nbdev.nbdev_export()